# Zero‑Shot, Few‑Shot, and CoT Prompting with Guardrails (Gemini API)

This notebook demonstrates advanced prompting strategies for large language models (LLMs) using the Gemini API in a marketing analytics context. The workflow includes:

- **Setup:** Connects to the Gemini API using a secure API key.
- **Sample Data:** Generates synthetic marketing KPI data for multiple channels.
- **Prompting Techniques:**
  - **Zero‑Shot Prompting:** Asks the model to summarize weekly marketing performance using only provided context, with strict guardrails for knowledge boundaries and refusal if data is insufficient.
  - **Few‑Shot Prompting:** Provides the model with example inputs and outputs (demos), including a red-flag case, to guide its summary generation for new data.
  - **Chain‑of‑Thought (CoT) Prompting:** Instructs the model to reason internally step-by-step (without revealing its reasoning) before outputting a concise summary, encouraging more robust and nuanced analysis.
- **Guardrails:** Enforces knowledge boundaries and refusal rules to ensure the model only uses provided data and responds appropriately to missing information.

> Set your `GOOGLE_API_KEY` (from Google AI Studio) in `Secrets` tab.

In [ ]:
from google.colab import userdata
try:
    import google.generativeai as genai
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    _GEMINI_READY = bool(userdata.get('GOOGLE_API_KEY'))
except Exception:
    print("Install google-generativeai to enable live calls.")
    _GEMINI_READY = False

In [ ]:
import json, textwrap, random
import pandas as pd
import numpy as np
from typing import Any, Dict, Tuple, List, Optional

MODEL_NAME = "gemini-2.5-flash-lite"

def get_model(temperature: float = 0.3):
    if not _GEMINI_READY:
        return None
    cfg = {"temperature": temperature}
    return genai.GenerativeModel(MODEL_NAME, generation_config=cfg)


## Sample KPI Data (Marketing Story)


In [ ]:
np.random.seed(1)
channels = ["Search","Social","Display","Affiliate","Email"]
kpi = pd.DataFrame({
    "channel": channels,
    "impressions": np.random.randint(1_000_000, 5_000_000, size=len(channels)),
    "clicks": np.random.randint(20_000, 120_000, size=len(channels)),
    "spend_usd": np.random.randint(10_000, 70_000, size=len(channels)),
    "conversions": np.random.randint(300, 2_500, size=len(channels)),
    "adstock_index": np.round(np.random.uniform(0.6, 1.3, size=len(channels)), 2),
})
kpi["ctr"] = (kpi["clicks"]/kpi["impressions"]).round(4)
kpi["cvr"] = (kpi["conversions"]/kpi["clicks"]).round(4).clip(0,1)
kpi["cpa"] = (kpi["spend_usd"]/kpi["conversions"]).round(2)
display(kpi)


## Helpers


In [ ]:
def kpi_to_xml(df: pd.DataFrame) -> str:
    rows = []
    for _,r in df.iterrows():
        row = "".join([f"<{c}>{r[c]}</{c}>" for c in df.columns])
        rows.append(f"<row>{row}</row>")
    return "<kpi>\n" + "\n".join(rows) + "\n</kpi>"

def build_notes_xml(text: str) -> str:
    return f"<notes>\n{text}\n</notes>"

def default_sales_notes() -> str:
    return ("Retail steady; enterprise slower. Affiliate bundle launched; "
            "email CTR strong but mobile checkout underperformed mid-week.")


## Zero‑Shot Prompt


In [ ]:
ZERO_SHOT_SYSTEM = textwrap.dedent("""
You are a marketing analytics assistant for weekly planning.
Use only the provided context. If required data are missing, return an insufficient information error.
""").strip()

def zero_shot_prompt(kpi_xml: str, notes_xml: str) -> str:
    return textwrap.dedent(f"""
    ROLE: Marketing analytics assistant
    INSTRUCTIONS: Summarize weekly performance in <=120 words (leadership-friendly).
    KNOWLEDGE BOUNDARY: Use ONLY the context below.
    If insufficient, output the refusal message (error, reason).

    OUTPUT: Brief summary in a plain text.

    CONTEXT START
    {kpi_xml}
    {notes_xml}
    CONTEXT END
    """).strip()

def run_zero_shot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.2):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(zero_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(temperature=temperature)
    prompt = zero_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content([ZERO_SHOT_SYSTEM, prompt])
    return resp.text

zero_shot_output = run_zero_shot(kpi)
print(zero_shot_output)


## Few‑Shot Prompt (with 2 Demos, incl. Red‑Flag Case)


In [ ]:
DEMO_1_INPUT = {
  "kpi":[
    {"channel":"Search","ctr":0.028,"cvr":0.032,"cpa":65,"adstock_index":1.05},
    {"channel":"Social","ctr":0.035,"cvr":0.018,"cpa":130,"adstock_index":0.90}
  ],
  "notes":"Mobile checkout bug fixed mid-week."
}
DEMO_1_OUTPUT = "Search efficient; Social high clicks but weak conversion."

DEMO_2_INPUT = {
  "kpi":[
    {"channel":"Display","ctr":0.012,"cvr":0.026,"cpa":95,"adstock_index":1.10},
    {"channel":"Email","ctr":0.055,"cvr":0.041,"cpa":40,"adstock_index":0.80}
  ],
  "notes":"Partner newsletter featuring bundles."
}
DEMO_2_OUTPUT = "Email strong engagement and conversion; Display stable."

def few_shot_prompt(kpi_xml: str, notes_xml: str) -> str:
    demos = textwrap.dedent(f"""
    DEMO 1 INPUT:
    <kpi_demo>{json.dumps(DEMO_1_INPUT["kpi"])}</kpi_demo>
    <notes_demo>{DEMO_1_INPUT["notes"]}</notes_demo>
    DEMO 1 OUTPUT:
    {DEMO_1_OUTPUT}

    DEMO 2 INPUT:
    <kpi_demo>{json.dumps(DEMO_2_INPUT["kpi"])}</kpi_demo>
    <notes_demo>{DEMO_2_INPUT["notes"]}</notes_demo>
    DEMO 2 OUTPUT:
    {DEMO_2_OUTPUT}
    """).strip()

    return textwrap.dedent(f"""
    ROLE: Marketing analytics assistant
    INSTRUCTIONS: Use only provided context. If insufficient, return refusal.
    OUTPUT: Brief summary in a plain text

    {demos}

    LIVE INPUT:
    {kpi_xml}
    {notes_xml}
    """).strip()

def run_few_shot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.2):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(few_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(temperature=temperature)
    prompt = few_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content(prompt)
    return resp.text

few_shot_output = run_few_shot(kpi)
print(few_shot_output)


## Chain‑of‑Thought (Internal) + Brief Rationale


In [ ]:
COT_INTERNAL = """
Reason step-by-step internally. Do NOT reveal your chain of thought.
Output only the brief summary with 1-3 bullets.
"""

def cot_prompt(kpi_xml: str, notes_xml: str) -> str:
    return textwrap.dedent(f"""
    ROLE: Senior marketing analyst
    INSTRUCTIONS:
    - {COT_INTERNAL.strip()}
    - Sanity-check conflicting signals (e.g., CTR up but CVR down) before deciding.
    OUTPUT: Brief summary in a plain text.

    CONTEXT:
    {kpi_xml}
    {notes_xml}
    """).strip()

def run_cot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.3):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(cot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(temperature=temperature)
    prompt = cot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content(prompt)
    return resp.text

cot_output = run_cot(kpi)
print(cot_output)